<a href="https://colab.research.google.com/github/abdulrehman898998/gpt2/blob/main/spam%20non%20pam%20classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install tiktoken
!pip install transformers datasets tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
df = pd.read_csv('/content/SMSSpamCollection', sep='\t', header=None ,names=['labels', 'text'] )
df

,labels,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
def balanced_data(df):
  spam_len=df[df['labels']=='spam'].shape[0]
  ham_df=df[df['labels']=='ham'].sample(spam_len, random_state=123)
  balanced_df = pd.concat([ham_df, df[df["labels"] == "spam"]])

  return balanced_df

balanced_df = balanced_data(df)
print(balanced_df["labels"].value_counts())


labels
ham     747
spam    747
Name: count, dtype: int64


In [ ]:
def train_test(balanced_df,train_ratio,val_ratio):
  train_data=balanced_df[:int(train_ratio*len(balanced_df))]
  test_data=balanced_df[int(train_ratio*len(balanced_df)):int((train_ratio+val_ratio)*len(balanced_df)+1)]
  val_data=balanced_df[int((train_ratio+val_ratio)*len(balanced_df)+1):]
  return train_data,val_data,test_data

In [ ]:
train_data,val_data,test_data=train_test(balanced_df,0.7,0.2)

In [ ]:
train_data.to_csv('/content/train_data.csv',index=False)
val_data.to_csv('/content/val_data.csv',index=False)
test_data.to_csv('/content/test_data.csv',index=False)

In [ ]:
len(train_data),len(val_data),len(test_data)

(1045, 149, 300)

df['target'].value_counts()

In [ ]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd

class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50255):
        data = pd.read_csv(csv_file)
        self.max_length = max_length


        self.labels = data["labels"].map({"ham": 0, "spam": 1}).values


        encoded_data = [tokenizer.encode(text) for text in data["text"]]


        if self.max_length is None:
            self.max_length = max(len(text) for text in encoded_data)

        else:
            self.max_length = min(self.max_length, max(len(text) for text in encoded_data))
        self.encoded_data = [
            text[:self.max_length] + [pad_token_id] * max(0, self.max_length - len(text[:self.max_length]))
            for text in encoded_data
        ]

    def __len__(self):
        return len(self.encoded_data)

    def __getitem__(self, idx):
        # Return encoded text and label as tensors
        return (
            torch.tensor(self.encoded_data[idx], dtype=torch.long),
            torch.tensor(self.labels[idx], dtype=torch.long),
        )


In [ ]:
train_dataset=SpamDataset(csv_file='/content/train_data.csv',tokenizer=tokenizer)
val_dataset=SpamDataset(csv_file='/content/val_data.csv',tokenizer=tokenizer)
test_dataset=SpamDataset(csv_file='/content/test_data.csv',tokenizer=tokenizer)

In [ ]:
train_dataset[0]

(tensor([   32,  2503,  4818,   318,  6029,     0,   775,   460,   892,   286,
          1223,   284,   466,   339,   339,     0,  8192,   257,  3621,   640,
          9975,  2801,  2192,   256,   742,   334,  1568,  8615,   545, 21757,
         36147,  2124,  5324,    13, 50255, 50255, 50255, 50255, 50255, 50255,
         50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255,
         50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255,
         50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255,
         50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255,
         50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255,
         50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255,
         50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255,
         50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255, 50255]),
 tensor(0))

In [ ]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pretrained GPT-2 model and tokenizer
model_name = "gpt2"  # You can use other variants like "gpt2-medium", "gpt2-large", or "gpt2-xl"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.config

GPT2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.46.2",
  "use_cache": true,
  "vocab_size": 50257
}

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


input_text = "who are humans"
input_ids = tokenizer.encode(input_text, return_tensors="pt")


output = model.generate(input_ids, max_length=50, num_return_sequences=1, do_sample=True)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated text: who are humans.) I will share with you why I am talking about my personal space and my own experience of humanism.

I am an experienced humanist. I have read some of George Orwell's works ("The Coming of the Human Race


In [ ]:
torch.manual_seed(123)

num_classes = 2
model.lm_head = torch.nn.Linear(in_features=768, out_features=num_classes)

In [ ]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=2, bias=True)
)


In [ ]:
def calculate_acc(model,device,data_loader,num_batches=None):
  model.eval()
  correct_pred=0
  example_seen=0
  if num_batches is None:
    num_batches=len(data_loader)
  else:
    num_batches=min(num_batches,len(data_loader))
  for i,(inputs,labels) in enumerate(data_loader):
    if i<num_batches:
      inputs=inputs.to(device)
      labels=labels.to(device)
      with torch.no_grad():
        outputs = model(inputs)
        logits = outputs.logits[:, -1, :]
      outputs=torch.argmax(logits,dim=-1)

      example_seen+=inputs.shape[0]
      correct_pred+=(outputs==labels).sum().item()

    else:
      break
  return correct_pred/example_seen

In [ ]:
def calculate_loss_batch(model,device,inputs,labels):
  model.to(device)
  inputs=inputs.to(device)
  labels=labels.to(device)
  outputs=model(inputs)
  logits = outputs.logits
  logits = logits[:, -1, :]
  loss=torch.nn.functional.cross_entropy(logits,labels)
  return loss

In [ ]:
def calculate_loss(model,device,data_loader,num_batches=None):
  total_loss=0

  if num_batches is None:
    num_batches=len(data_loader)
  else:
    num_batches=min(num_batches,len(data_loader))
  for i,(inputs,labels) in enumerate(data_loader):
    if i<num_batches:
      loss=calculate_loss_batch(model,device,inputs,labels).item()
      total_loss+=loss



    else:
      break
  return total_loss/num_batches

In [ ]:
def eval_model(model,train_loader,val_loader,device,eval_iter):
  model.eval()
  train_loss=calculate_loss(model,device,train_loader,num_batches=eval_iter)
  val_loss=calculate_loss(model,device,val_loader,num_batches=eval_iter)
  model.train()
  return train_loss,val_loss

In [ ]:
def training(model,num_epochs,optimizer,eval_freq,train_loader,val_loader,device,eval_iter):
  examples_seen=0
  global_step=-1
  train_losses,val_losses=[],[]
  train_accs, val_accs = [], []
  for epoch in range(num_epochs):
    model.train()
    for inputs,labels in train_loader:
      optimizer.zero_grad()
      loss=calculate_loss_batch(model, device, inputs, labels)
      loss.backward()
      optimizer.step()
      examples_seen+=inputs.shape[0]
      global_step+=1

      if global_step % eval_freq==0:
        train_loss,val_loss=eval_model(model, train_loader, val_loader, device, eval_iter)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print(f"Ep {epoch+1} (Step {global_step:06d}): "
              f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")
    train_accuracy=calculate_acc(model,device,train_loader,num_batches=eval_iter)
    val_accuracy=calculate_acc(model,device,val_loader,num_batches=eval_iter)
    print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
    print(f"Validation accuracy: {val_accuracy*100:.2f}%")
    train_accs.append(train_accuracy)
    val_accs.append(val_accuracy)
  return train_losses,val_losses,train_accs, val_accs,examples_seen

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import time

start_time = time.time()

torch.manual_seed(123)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)


train_losses, val_losses, train_accs, val_accs, examples_seen = training(
    model, 5, optimizer, 50, train_loader, val_loader, device, 5  # num_epochs=5, eval_freq=50, eval_iter=5
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 0.535, Val loss 0.849
Ep 1 (Step 000050): Train loss 0.052, Val loss 0.090
Ep 1 (Step 000100): Train loss 0.169, Val loss 0.124
Training accuracy: 100.00% | Validation accuracy: 95.00%
Ep 2 (Step 000150): Train loss 0.035, Val loss 0.075
Ep 2 (Step 000250): Train loss 0.018, Val loss 0.166
Training accuracy: 100.00% | Validation accuracy: 95.00%
Ep 3 (Step 000300): Train loss 0.001, Val loss 0.233
Ep 3 (Step 000350): Train loss 0.000, Val loss 0.209
Training accuracy: 95.00% | Validation accuracy: 95.00%
Ep 4 (Step 000400): Train loss 0.012, Val loss 0.181
Ep 4 (Step 000450): Train loss 0.000, Val loss 0.326
Ep 4 (Step 000500): Train loss 0.057, Val loss 0.013
Training accuracy: 100.00% | Validation accuracy: 95.00%
Ep 5 (Step 000550): Train loss 0.000, Val loss 0.177
Ep 5 (Step 000600): Train loss 0.001, Val loss 0.104
Training accuracy: 97.50% | Validation accuracy: 100.00%
Training completed in 2.45 minutes.


In [ ]:
def classify_review(text, model, tokenizer, device, max_length=None, pad_token_id=50256):
    model.eval()

    input_ids = tokenizer.encode(text)
    input_ids += [pad_token_id] * (max_length - len(input_ids))
    input_tensor = torch.tensor(input_ids, device=device).unsqueeze(0)

    with torch.no_grad():
        # Assuming the logits are the first element in the tuple returned by the model
        outputs = model(input_tensor)
        #If the output is a tuple, and logits are at index 0
        logits = outputs[0][:, -1, :]
        #If the output is a dictionary, and logits are under the key 'logits'
        #logits = outputs.logits[:, -1, :]

    predicted_label = torch.argmax(logits, dim=-1).item()

    return "spam" if predicted_label == 1 else "not spam"

In [ ]:
text_1 = (
"We are pleased to announce that regular academic activities will resume in Face-to-Face (FTF) mode at all H-12 institutions, CoEME, and MCS, effective Thursday, November 28, 2024."

# Regarding makeup classes, a separate plan will be shared later for Saturday, November 30, 2024.'
# )
# )
)


print(classify_review(
    text_1, model, tokenizer, device, max_length=train_dataset.max_length
))

not spam


In [ ]:
import torch

# Save the model weights
model_save_path = "/content/fine_tuned_gpt2.pth"
torch.save(model.state_dict(), model_save_path)

# Save the tokenizer (important for GPT-2 models)
tokenizer_save_path = "/content/gpt2_tokenizer"
model.save_pretrained(tokenizer_save_path)


In [ ]:
!pip install huggingface_hub

# Step 1: Log in to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

# Step 2: Delete old repository directory or clone to a new one
import shutil
import os

# Delete old directory: (Recommended)
if os.path.exists("model_repo"):
    shutil.rmtree("model_repo")

# Step 3: Clone the repository from Hugging Face
from huggingface_hub import Repository

# Set your Hugging Face username and repository name
username = "abdulrehman89OK"  # Replace with your username
repo_name = "spam_non_sapm_classifier"  # Replace with your repository name
repo_url = f"https://huggingface.co/{username}/{repo_name}"

# Clone the repository to the Colab environment
repo = Repository(local_dir="model_repo", clone_from=repo_url)

# Step 3: Verify that model and tokenizer files exist in Colab
import os

model_file = "/content/fine_tuned_gpt2.pth"  # Path to your saved model file
tokenizer_dir = "/content/gpt2_tokenizer"   # Path to your tokenizer folder

if not os.path.exists(model_file):
    raise FileNotFoundError(f"Model file not found at {model_file}")
if not os.path.exists(tokenizer_dir):
    raise FileNotFoundError(f"Tokenizer directory not found at {tokenizer_dir}")

# Step 4: Copy the model file to the repository
shutil.copy(model_file, "model_repo/pytorch_model.bin")  # Rename to pytorch_model.bin

# Step 5: Copy tokenizer files to the repository
shutil.copytree(tokenizer_dir, "model_repo/tokenizer", dirs_exist_ok=True)

# Step 6: (Optional) Add a config.json file for the model
# Create a config.json file if you have additional configurations for the classification head
config_path = "model_repo/config.json"
config = {
    "model_type": "gpt2",
    "num_labels": 2,  # Number of classification labels (spam and ham)
    "id2label": {0: "ham", 1: "spam"},
    "label2id": {"ham": 0, "spam": 1},
}
import json
with open(config_path, "w") as f:
    json.dump(config, f)

# Step 7: Configure Git and push the repository
!git config --global user.email "abdulrehmanpti12@gmail.com"  # Replace with your email
!git config --global user.name "abdulrehman89OK"             # Replace with your username

# Commit and push files to Hugging Face
repo.push_to_hub(commit_message="Initial commit of fine-tuned GPT-2 with classification head and tokenizer")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/abdulrehman89OK/spam_non_sapm_classifier into local empty directory.


Upload file pytorch_model.bin:   0%|          | 1.00/475M [00:00<?, ?B/s]

Upload file tokenizer/model.safetensors:   0%|          | 1.00/475M [00:00<?, ?B/s]

To https://huggingface.co/abdulrehman89OK/spam_non_sapm_classifier
   720a8cd..39ce82c  main -> main

   720a8cd..39ce82c  main -> main



'https://huggingface.co/abdulrehman89OK/spam_non_sapm_classifier/commit/39ce82cfe58735bf15b39fe364b6c1cfb85cfdd9'